<a href="https://colab.research.google.com/github/Monisha-11/EXP--06-MODERN-WEB/blob/main/Monte_Carlo(WORKSHOP).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [59]:
pip install git+https://github.com/mimoralea/gym-walk#egg=gym-walk

  Cloning https://github.com/mimoralea/gym-walk to /tmp/pip-install-p8lakh_t/gym-walk_dedc7f8bda714b46a066047c4fc98cc4
  Running command git clone --filter=blob:none --quiet https://github.com/mimoralea/gym-walk /tmp/pip-install-p8lakh_t/gym-walk_dedc7f8bda714b46a066047c4fc98cc4
  Resolved https://github.com/mimoralea/gym-walk to commit 5999016267d6de2f5a63307fb00dfd63de319ac1
  Preparing metadata (setup.py) ... done


In [60]:
import warnings ; warnings.filterwarnings('ignore')

import gym, gym_walk
import numpy as np

import random
import warnings

warnings.filterwarnings('ignore', category=DeprecationWarning)
np.set_printoptions(suppress=True)
random.seed(123); np.random.seed(123)

# Reference https://github.com/mimoralea/gym-walk

In [61]:
# SLIPPERY WALK

env = gym.make('SlipperyWalkFive-v0')
P = env.env.P
init_state = env.reset()
goal_state = 6
LEFT, RIGHT = range(2)

In [62]:
P

{0: {0: [(0.5000000000000001, 0, 0.0, True),
   (0.3333333333333333, 0, 0.0, True),
   (0.16666666666666666, 0, 0.0, True)],
  1: [(0.5000000000000001, 0, 0.0, True),
   (0.3333333333333333, 0, 0.0, True),
   (0.16666666666666666, 0, 0.0, True)]},
 1: {0: [(0.5000000000000001, 0, 0.0, True),
   (0.3333333333333333, 1, 0.0, False),
   (0.16666666666666666, 2, 0.0, False)],
  1: [(0.5000000000000001, 2, 0.0, False),
   (0.3333333333333333, 1, 0.0, False),
   (0.16666666666666666, 0, 0.0, True)]},
 2: {0: [(0.5000000000000001, 1, 0.0, False),
   (0.3333333333333333, 2, 0.0, False),
   (0.16666666666666666, 3, 0.0, False)],
  1: [(0.5000000000000001, 3, 0.0, False),
   (0.3333333333333333, 2, 0.0, False),
   (0.16666666666666666, 1, 0.0, False)]},
 3: {0: [(0.5000000000000001, 2, 0.0, False),
   (0.3333333333333333, 3, 0.0, False),
   (0.16666666666666666, 4, 0.0, False)],
  1: [(0.5000000000000001, 4, 0.0, False),
   (0.3333333333333333, 3, 0.0, False),
   (0.16666666666666666, 2, 0.0, Fa

In [63]:
def print_policy(pi, P, action_symbols=('<', 'v', '>', '^'), n_cols=4, title='Policy:'):
    print(title)
    arrs = {k:v for k,v in enumerate(action_symbols)}
    for s in range(len(P)):
        a = pi(s)
        print("| ", end="")
        if np.all([done for action in P[s].values() for _, _, _, done in action]):
            print("".rjust(9), end=" ")
        else:
            print(str(s).zfill(2), arrs[a].rjust(6), end=" ")
        if (s + 1) % n_cols == 0: print("|")

In [64]:
# Adversarial Policy
pi_1 = lambda s: {
    0:LEFT, 1:LEFT, 2:LEFT, 3:LEFT, 4:LEFT, 5:LEFT, 6:LEFT
}[s]
print_policy(pi_1, P, action_symbols=('<', '>'), n_cols=7)

Policy:
|           | 01      < | 02      < | 03      < | 04      < | 05      < |           |


##    **DECAY_SCHEDULE**

In [65]:
def decay_schedule(init_value,min_value,decay_ratio,max_steps,log_start=-2,log_base=10):
  decay_steps = int(max_steps * decay_ratio)
  rem_steps = max_steps - decay_steps
  values = np.logspace(log_start,0,decay_steps,base=log_base,endpoint=True)[::-1]
  values = (values - values.min())/(values.max()-values.min())
  values = (init_value - min_value)*values+min_value
  values = np.pad(values,(0,rem_steps),'edge')
  return values

## **GENERATE_TAJECTORY**

In [66]:
from itertools import count
def generate_trajectory(pi_1,P,max_steps=20):
  done,trajectory = False,[]
  while not done:
    state = env.reset()
    for t in count():
      action = pi_1(state)
      next_state,reward,done,_ = env.step(action)
      experience = (state,action,reward,next_state,done)
      trajectory.append(experience)
      if done:
        break
      if t>=max_steps - 1:
        trajectory = []
        break
      s = next_state
  return np.array(trajectory,np.object)


## **MC_PREDICTION**

In [67]:
from tqdm import tqdm
def mc_prediction(pi_1,P,gamma=1.0,init_alpha=0.5,min_alpha=0.01,alpha_decay_ratio=0.3,n_episodes=500,max_steps=100,first_visit=True):
  nS=env.observation_space.n
  discounts = np.logspace(0,max_steps,num=max_steps,base=gamma,endpoint=False)
  alphas = decay_schedule(init_alpha,min_alpha,alpha_decay_ratio,n_episodes)
  V = np.zeros(nS)
  V_track = np.zeros((n_episodes,nS))
  for e in tqdm(range(n_episodes),leave=False):
    trajectory=generate_trajectory(pi_1,P,max_steps)
    visited = np.zeros(nS,dtype=np.bool)
    for t,(state,_,reward,_,_)in enumerate(trajectory):
      if visited[state]and first_visit:
        continue
      visited[state]=True
      n_steps = len(trajectory[t:])
      G = np.sum(discounts[:n_steps]*trajectory[t:,2])
      V[state]=V[state]+alphas[e] * (G-V[state])
      V_track[e]=V
  return V.copy()

## **OUTPUT OF MC_PREDICTION**

In [68]:
print(mc_prediction(pi_1,P))

[0.         0.         0.         0.04269423 0.         0.
 0.        ]


## **Comparison of Monte Carlo output with Dynamic programming output**

In [69]:
def policy_evaluation(pi, P, gamma=1.0, theta=1e-10):
    prev_V = np.zeros(len(P), dtype=np.float64)
    # Write your code here to evaluate the given policy
    while True:
      V = np.zeros(len(P))
      for s in range(len(P)):
        for prob, next_state, reward, done in P[s][pi(s)]:
          V[s] += prob * (reward + gamma *  prev_V[next_state] * (not done))
      if np.max(np.abs(prev_V - V)) < theta:
        break
      prev_V = V.copy()
    return V

In [70]:
def print_state_value_function(V, P, n_cols=4, prec=3, title='State-value function:'):
    print(title)
    for s in range(len(P)):
        v = V[s]
        print("| ", end="")
        if np.all([done for action in P[s].values() for _, _, _, done in action]):
            print("".rjust(9), end=" ")
        else:
            print(str(s).zfill(2), '{}'.format(np.round(v, prec)).rjust(6), end=" ")
        if (s + 1) % n_cols == 0: print("|")

In [71]:
V1 = policy_evaluation(pi_1, P)

In [72]:
V2 = mc_prediction(pi_1, P)

In [73]:
V1

array([0.        , 0.00274725, 0.01098901, 0.03571429, 0.10989011,
       0.33241758, 0.        ])

In [78]:
V2

array([0.       , 0.       , 0.       , 0.0423608, 0.       , 0.       ,
       0.       ])

In [79]:
V1>V2

array([False,  True,  True, False,  True,  True, False])

In [80]:
V2>V1

array([False, False, False,  True, False, False, False])

In [82]:
if(np.sum(V1>=V2)>=5):
  print("The first Method is better")
elif(np.sum(V2>=V1)>=5):
  print("The second method is better")
else:
  print("Both method have their merits.")

The first Method is better
